In [1]:
IN1 = !cat aoc/2023/day1

In [2]:
digits = lambda s: [(i, c) for i, c in enumerate(s) if str.isdigit(c)]
first_and_last = lambda a: (a[0][1], a[-1][1])
get_num = lambda ds: int("".join(first_and_last(ds)))

ans1 = sum(get_num(digits(s)) for s in IN1)

########################################

ws = {"one": "1", "two": "2", "three": "3", "four": "4", "five": "5", "six": "6", "seven": "7", "eight": "8", "nine" : "9"}

def unword(s, items): 
    for it in items:
        s = s.replace(*it)
    return digits(s)

# No digit word can overlap with itself, so the first and last word
# will be unharmed in at least one of the replacement directions
digits_both_ways = lambda s: sorted(unword(s, ws.items()) + unword(s, reversed(ws.items())))

ans2 = sum(get_num(digits_both_ways(s)) for s in IN1)

ans1, ans2

(55090, 54845)

In [3]:
IN2 = !cat aoc/2023/day2
IN2 = [r.split(": ")[1] for r in IN2]

In [4]:
from functools import reduce
from operator import mul, le

BAG = {"red": 12, "green": 13, "blue": 14}

games = lambda r: [dict(reversed(x.split(" ")) for x in ii.strip().split(", ")) for ii in r.split(";")]
vals = lambda g: [int(g.get(col, 0)) for col in BAG]
maxv = lambda gs: map(max, *map(vals, gs))

ans1 = sum((i+1) for i, r in enumerate(IN2) if all(map(le, maxv(games(r)), vals(BAG))))

########################################

power = lambda v: reduce(mul, v, 1)

ans2 = sum(power(maxv(games(r))) for r in IN2)

ans1, ans2

(2716, 72227)

In [5]:
IN3 = !cat aoc/2023/day3

In [6]:
import re

M, N = len(IN3), len(IN3[0])
board = [(i, j) for i in range(M) for j in range(N)]
x3x3 = lambda p: [(p[0]+x, p[1]+y) for x in [-1, 0, 1] for y in [-1, 0, 1]]
val = lambda p: IN3[p[0]][p[1]]

valid = lambda cs: (0, 0) <= cs <= (M-1, N-1)

neighb = lambda p: {q for q in x3x3(p) if valid(q)}
fig_neighb = lambda fig: set.union(*[neighb(p) for p in fig])

all_sym = {p for p in board if val(p) not in "0123456789."}

row_nums = lambda r: [(range(*m.span()), m.group()) for m in re.finditer("([0-9]+)", r)]
row_neighbs = lambda i, r: [(num, fig_neighb((i, j) for j in span)) for span, num in row_nums(r)]
all_nums = sum([row_neighbs(i, r) for i, r in enumerate(IN3)], [])

ans1 = sum(int(n) for n, nb in all_nums if nb & all_sym)

########################################

all_gears = {p for p in board if val(p) == "*"}
gear_nums = [[int(n) for n, nb in all_nums if p in nb] for p in all_gears]

ans2 = sum(v[0]*v[1] for v in gear_nums if len(v) == 2)
                
ans1, ans2            

(520019, 75519888)

In [7]:
IN4 = !cat aoc/2023/day4

In [8]:
cards = [[set(s.split()) for s in r.split("|")] for r in IN4]
counts = [len(w & n) for w, n in cards]

ans1 = sum(int(2**(c-1)) for c in counts)

########################################

from functools import reduce

ncopy = lambda its: 1 + sum(it.pop() for it in its if it)
win = lambda t, c: [t[0] + (won:=ncopy(t[1])), t[1] + [[won]*c]]

ans2 = reduce(win, counts, (0, []))[0]

#### Readable immutable version of part 2

total, clone_lists = 0, []
for count in counts:
    copies_of_n = 1 + sum(cl[0] for cl in clone_lists if cl)
    total += copies_of_n
    
    clone_lists = [cl[1:] for cl in clone_lists if cl[1:]] + [[copies_of_n] * count]
    
ans2_1 = total

ans1, ans2, ans2_1

(21821, 5539496, 5539496)

In [9]:
IN5 = !cat aoc/2023/day5

In [10]:
from itertools import groupby
from functools import reduce

seeds = [int(x) for x in IN5[0].split()[1:]]

parse_map = lambda g: [[int(x) for x in s.split()] for s in list(g)[1:]]
maps = [parse_map(g) for k, g in groupby(IN5[2:], key=bool) if k]

in_range = lambda r, x: r[0] <= x < r[0]+r[-1]
fwd_map  = lambda p, m: next((p-r[1]+r[0] for r in m if in_range(r[1:], p)), p)
fwd_list = lambda ps, m: [fwd_map(p, m) for p in ps]
end_pos  = lambda ps: reduce(fwd_list, maps, ps)

ans1 = min(end_pos(seeds))

########################################

back_map = lambda p, m: next((p-r[0]+r[1] for r in m if in_range(r, p)), p)
back_list = lambda ps, m: [back_map(p, m) for p in ps]

# There's probably some better dynamic programming algo.
# The main insight is that for a given starting range,
# the minimum will be achieved either at its starting point,
# or at one of the starting points of the intermediate map ranges.
# To prove, assume the absolute min happens at K, which does not
# map to any starting point at any stage - but then K-1 is
# guaranteed to be in the same range as K on every map, including the starting one
# and ends up one lower at the end, a contradiction.
# So we pull all starting points back to the seed level,
# and then check en masse for their ending positions.

seed_ranges = list(zip(seeds[::2], seeds[1::2]))

map_starts = lambda m: [r[1] for r in m]
pull = lambda prev_starts, m: back_list(prev_starts, m) + map_starts(m)
pre_starts = reduce(pull, reversed(maps), [])

ends = dict(zip(pre_starts, end_pos(pre_starts)))

ends_for_range = lambda r: [ends[s] for s in pre_starts if in_range(r, s)]
min_for_range = lambda r: min(end_pos(r) + ends_for_range(r))
ans2 = min(min_for_range(r) for r in seed_ranges)

ans1, ans2

(51752125, 12634632)

In [11]:
IN6 = !cat aoc/2023/day6

In [12]:
races = [[int(x) for x in r.split()[1:]] for r in IN6]
root = lambda t,d: t/2 + (t*t/4 - d) ** 0.5 - 0.001
wins = lambda p, r: p * (int(root(*r)) - int(r[0]-root(*r)))

ans1 = reduce(wins, zip(*races), 1)

########################################

race = [int("".join(r.split()[1:])) for r in IN6]
ans2 = wins(1, race)

ans1, ans2

(252000, 36992486)

In [13]:
IN7 = !cat aoc/2023/day7


In [14]:
from collections import Counter
order = "23456789TJQKA"

htype = lambda s: sorted(Counter(s).values(), reverse=True)
hand = lambda s: [order.find(c) for c in s]

hands = lambda typer: [(typer(s), hand(s), int(bid)) for s, bid in map(str.split, IN7)]
winnings = lambda typer: sum((i+1)*h[2] for i, h in enumerate(sorted(hands(typer))))

ans1 = winnings(htype)

########################################

order = "J23456789TQKA"

noj = lambda s: htype(s.replace("J", "")) or [0]
jtype = lambda s: [s.count("J") + noj(s)[0]] + noj(s)[1:]

ans2 = winnings(jtype)

ans1, ans2

(248453531, 248781813)

In [15]:
IN8 = !cat aoc/2023/day8

In [16]:
from itertools import cycle
from math import lcm

parse = lambda rows: ([c=='R' for c in rows[0]], {r[:3]: [r[7:10], r[12:15]] for r in rows[2:]})
instr, dirs = parse(IN8)

fsm = lambda state: ((i+1, (state := dirs[state][d])) for i, d in enumerate(cycle(instr)))
steps_between = lambda start, term: next(i for i, x in fsm(start) if (term(x) if callable(term) else term==x))

ans1 = steps_between("AAA", "ZZZ")

########################################

# This is a very well-behaved fsm, the instruction list has prime length
# and each -A orbit contains exactly one -Z, so we just find the least common multiple
# This will not work on an arbitrary set of terminal states, or a more complicated graph

endswith = lambda c: lambda s: s.endswith(c)
ans2 = lcm(*[steps_between(k, endswith("Z")) for k in dirs if endswith("A")(k)])

ans1, ans2

(22411, 11188774513823)

In [17]:
IN9 = !cat aoc/2023/day9

In [18]:
diff = lambda seq: [y-x for y, x in zip(seq[1:], seq[:-1])]
rows = [list(map(int, r.split())) for r in IN9]
def extrapolate(seq):
    if not any(seq):
        return 0
    return seq[-1] + extrapolate(diff(seq))

ans1 = sum(map(extrapolate, rows))

########################################

extrapolate_front = lambda seq: -extrapolate([-x for x in reversed(seq)])

ans2 = sum(map(extrapolate_front, rows))

ans1, ans2

(1681758908, 803)

In [19]:
IN10 = !cat aoc/2023/day10
BIN10 = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""".split("\n")

In [20]:
from itertools import product
from operator import neg
board = IN10

sym = lambda p: board[p[0]][p[1]]
plus = lambda *ps: sum(ps)
add = lambda *ps: tuple(map(plus, *ps)) or (0,0)
sub = lambda p, q: add(p, map(neg, q))

nbs = { }

conn = {
    "F": [(1, 0), (0, 1)], 
    "7": [(1, 0), (0, -1)], 
    "L": [(-1, 0), (0, 1)], 
    "J": [(-1, 0), (0, -1)],
}
path = []
for p in product(range(len(board)), range(len(board[0]))):
    nbs[p] = [add(p, c) for c in conn.get(sym(p), [])]

    if sym(p) == "S":
        path.append(p)

# First neighbor is annoying, since we have to detect that it points to the start.
path.append(next(p for p, ns in nbs.items() if path[0] in ns))

# After that the delta leading to the current cell will cancel out with one of its paths,
# and the remaining one will lead to the next cell in the path, so we just add them.
# We don't need to do anything for - or |, since the same vector leads in and out.
next_p = lambda p1, p2: add(p2, sub(p2, p1), *conn.get(sym(p2), []))

while path[0] != path[-1]:
    path.append(next_p(*path[-2:]))

ans1 = (len(path) // 2)

########################################

# Shoelace formula
area = abs(sum((p[0]*q[1] - p[1]*q[0]) for p, q in zip(path[:-1], path[1:]))) / 2

# Pick's theorem: interior = area - boundary/2 + 1
ans2 = int(area + 1 - ans1)


ans1, ans2

(7066, 401)

In [21]:
IN11 = !cat aoc/2023/day11
BIN11 = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....""".split("\n")

In [22]:
universe = IN11
rows = [0] * len(universe)
cols = [0] * len(universe[0])

for i, j in product(range(len(rows)), range(len(cols))):
    if universe[i][j] == "#":
        rows[i] += 1
        cols[j] += 1

def dist(arr, exp=2):
    tot = 0
    for i, v in enumerate(arr):
        if not v:
            continue
        d = 0
        for w in arr[i+1:]:
            d += 1
            if w:
                tot += d * v * w
            else:
                d += exp-1
    return tot
            
ans1 = dist(rows) + dist(cols)

ans2 = dist(rows, exp=1000000) + dist(cols, exp=1000000)

ans1, ans2

(9536038, 447744640566)

In [23]:
IN12 = !cat aoc/2023/day12
BIN12 = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split("\n")

In [24]:
%%time
from functools import lru_cache

@lru_cache
def arrangements(rec, conds):
    if not rec:
        return not any(conds)
    
    if not conds:
        return not any(c=="#" for c in rec)
    
    # Prune the recursion tree by rejecting early if not enough ?/#s for match, 
    # or if equal, make sure it creates the exact same runs as the conditions.
    # With the cache and this check things speed up significantly.
    if sum(conds) >= sum(c!="." for c in rec):
        runs = tuple(len(r) for r in rec.split(".") if r)
        return runs == conds
    
    # Arrangements are split on if we fulfill the first condition immediately or not.
    k = conds[0]
    if any(c == "." for c in rec[:k]) or rec[k] == "#":
        with_start = 0
    else:
        with_start = arrangements(rec[k+1:], conds[1:])

    if rec[0] == "#":
        no_start = 0
    else:
        no_start = arrangements(rec[1:], conds)
    
    return with_start + no_start

unfold = lambda rec, conds: ("?".join([rec]*5), conds*5)

ans1, ans2 = 0, 0
for r in IN12:
    rec, cs = r.split()
    conds = tuple(int(x) for x in cs.split(","))
    
    ans1 += arrangements(rec, conds)
    ans2 += arrangements(*unfold(rec, conds))

ans1, ans2

CPU times: user 419 ms, sys: 1.61 ms, total: 420 ms
Wall time: 424 ms


(7718, 128741994134728)

In [25]:
IN13 = !cat aoc/2023/day13
BIN13 = """
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#""".split("\n")

In [26]:
from collections import Counter
from itertools import groupby, product, chain

figs = [list(rs) for g, rs in groupby(IN13, bool) if g]

transpose = lambda fig: [[r[k] for r in fig] for k in range(len(fig[0]))]

rejected = lambda r: {(i, j): (i + j)/2 for i, j in product(range(len(r)), repeat=2) if i<j and r[i] != r[j]}
reject_all = lambda fig: set.union(*[set(rejected(r).values()) for r in fig])
mirror_col = lambda fig: next((k+1 for k in range(len(fig[0])-1) if k+0.5 not in reject_all(fig)), 0)
score = lambda mfunc, fig: mfunc(fig) or 100 * mfunc(transpose(fig))

ans1 = sum(score(mirror_col, f) for f in figs)

########################################

# Approach is good - find the only column rejected by a single cell pair.
# The code is bad, we should combine with previous stage to clean up.
mirror_with_smudge = lambda fig: next(
    (int(k+0.5) 
     for k, v in 
     Counter(chain(*[rejected(r).values() for r in fig])).items() 
     if k != int(k) and v == 1
    ), 0)

ans2 = sum(score(mirror_with_smudge, f) for f in figs)

ans1, ans2

(30518, 36735)

In [27]:
IN14 = !cat aoc/2023/day14
BIN14 = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....""".split("\n")

In [28]:
from collections import defaultdict

fig = IN14
as_str = lambda seq: "".join(seq)

tilted_up = lambda col: "#".join(as_str(sorted(grp, reverse=True)) for grp in col.split("#"))

column_load = lambda col: sum(len(col) - k for k, c in enumerate(col) if c == "O")
total_load = lambda cols: sum(column_load(col) for col in cols)

to_cols = lambda rows: [as_str(r[c] for r in rows) for c in range(len(rows[0]))]

ans1 = total_load(tilted_up(col) for col in to_cols(fig))

########################################

# All of this would be horribly inefficient if we had to do real 1000000000 cycles.
# Fortunately, our process loops at cycles 103->112, so we can just find the step 
# where it will end up mod 9, rather than have to compute each cycle.
# Rolling these balls seems Turing-complete (can have arbitrarily long stacks that only
# release one ball per cycle, and can add from another stack if non-empty), so
# there's absolutely no guarantee that this will finish in reasonable time or space,
# but it does for this input, as I guess it pretty much has to.
tilt_and_rotate = lambda fig: [tilted_up(col)[::-1] for col in to_cols(fig)]

def cycle(fig, n=1):
    seen = defaultdict(list)
    for k in range(n):
        start = tuple(fig)
        seen[start].append(k)
        
        if len(seen[start]) > 1:
            print(f"stable at {seen[start]}")
            break
        
        for _ in range(4):
            fig = tilt_and_rotate(fig) 
    else:
        return fig
    
    visits = seen[tuple(fig)]   
    final_idx = visits[0] + (n - visits[0]) % (visits[1] - visits[0])
    actual = next(k for k, v in seen.items() if final_idx in v)
    
    return actual


cycled_fig = cycle(fig, 1000000000)
ans2 = total_load(to_cols(cycled_fig))

ans1, ans2

stable at [103, 112]


(113424, 96003)

In [29]:
IN15 = !cat aoc/2023/day15
BIN15 = ["rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7"]


In [30]:
from collections import defaultdict
from functools import reduce

ops = IN15[0].split(",")

hasher = lambda v, c: 17*(v+ord(c)) % 256
HASH = lambda s: reduce(hasher, s, 0)

ans1 = sum(HASH(s) for s in ops)

########################################

boxes = defaultdict(dict)

for cmd in ops:
    label, *power = cmd.strip("-").split("=")
    box = boxes[HASH(label)]
    if power:
        box[label] = int(power[0])
    else:
        box.pop(label, None)
        

focus = lambda box: sum((i+1)*v for i, (k, v) in enumerate(box.items()))

ans2 = sum((k+1)*focus(v) for k, v in boxes.items())

ans1, ans2

(519603, 244342)

In [31]:
IN16 = !cat aoc/2023/day16
BIN16 = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....""".split("\n")

In [32]:
%%time
from functools import lru_cache

board = IN16
M, N = len(board), len(board[0])

sym = lambda p: board[p[0]][p[1]] if 0<= p[0] < M and 0 <= p[1] < N else None
step = lambda p, v: (p[0] + v[0], p[1] + v[1])

def redirect(v, sym):  
    if sym == "/":
        yield (-v[1], -v[0])
    elif sym == "\\":
        yield (v[1], v[0])
    elif sym == "|" and v[1]:
        yield from ((1, 0), (-1, 0))
    elif sym == "-" and v[0]:
        yield from ((0, 1), (0, -1))
    else:
        yield v

def energized(start_p, start_v):
    beams = [(start_p, start_v)]

    visited = set()

    while beams:
        p, v = beams.pop()
        if sym(p) and (p, v) not in visited:
            visited.add((p, v))
            for w in redirect(v, sym(p)):
                beams.append((step(p, w), w))

    return len({p for p, v in visited})

ans1 = energized((0, 0), (0, 1))

########################################


starts = (
    [((k, 0), (0, 1)) for k in range(M)] 
    + [((k, N-1), (0, -1)) for k in range(M)]
    + [((0, k), (1, 0)) for k in range(N)]
    + [((M-1, k), (-1, 0)) for k in range(N)]
)

ans2 = max(energized(p, v) for p, v in starts)

ans1, ans2

CPU times: user 3.17 s, sys: 7.77 ms, total: 3.18 s
Wall time: 3.21 s


(7979, 8437)

In [33]:
IN17 = !cat aoc/2023/day17

BIN17 = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533""".split("\n")

In [34]:
%%prun
from heapq import heappop, heappush
from collections import defaultdict

img = IN17
M = len(img)
N = len(img[0])

def move(pt, mv):
    return (pt[0] + mv[0], pt[1] + mv[1])

def valid(pt):
    return 0 <= pt[0] < M and 0 <= pt[1] < N

def neighbors(pt, mv, k):
    left = (-mv[1], mv[0])
    right = (mv[1], -mv[0])
    nb = {(move(pt, left), left, 1), (move(pt, right), right, 1)}

    if k < 3:
        nb.add((move(pt, mv), mv, k+1))

    return [n for n in nb if valid(n[0])]

def neighbors2(pt, mv, k):
    left = (-mv[1], mv[0])
    right = (mv[1], -mv[0])
    
    nb = set()
    
    if k < 10:
        if k >= 3:
            nb.add((move(pt, mv), mv, k+1))
        else:
            d = 4 - k
            nb.add((move(pt, (d*mv[0], d*mv[1])), mv, 4))
        
    if k >= 4:
        nb |= {(move(pt, left), left, 1), (move(pt, right), right, 1)}

    return [n for n in nb if valid(n[0])]

def reached(v, end):
    return v[0] == end

def reached2(v, end):
    return v[0] == end and v[2] >= 4

def cost(v):
    return int(img[v[0][0]][v[0][1]])


def pop_min(q):
    q.sort()
    return q.pop(0)


def dijkstra(img, start, end, nbfunc=neighbors, termfunc=reached):
    g = defaultdict(list)
    
    v0 = (start, (0, 1), 0)
    q, seen, mins = [(0, v0, ())], set(), {v0: 0}
    
    results = []
    
    while q:
        cost_to_v1, v1, path_to_v1 = heappop(q) # pop_min(q) # 
        if v1 not in seen:
            seen.add(v1)
            path_with_v1 = (*path_to_v1, v1)
            if termfunc(v1, end):
                return (cost_to_v1, path_with_v1)

            for v2 in nbfunc(*v1):
                if v2 in seen:
                    continue
                prev = mins.get(v2, None)
                
                cost_with_v2 = cost(v2) + cost_to_v1
                if prev is None or cost_with_v2 < prev:
                    mins[v2] = cost_with_v2
                    heappush(q, (cost_with_v2, v2, path_with_v1))
                    # q.insert(0, (cost_with_v2, v2, path_with_v1))


ans1, path1 = dijkstra(img, (0, 0), (M-1, N-1))

########################################

ans2, path2 = dijkstra(img, (0, 0), (M-1, N-1), neighbors2, reached2)

ans1, ans2

In [35]:
arrow = {
    (0, 1): ">",
    (1, 0): "v",
    (0, -1): "<",
    (-1, 0): "^",
}

def display(img, path):
    rows = []
    for i in range(M):
        cells = []
        for j in range(N):
            if v := next((v for v in path if v[0] == (i, j)), None):
                cells.append(arrow[v[1]])
            else:
                cells.append(img[i][j])
        rows.append("".join(cells))
    return "\n".join(rows)

print(display(img, path2))

>253>4343122414426341311521355436756516735365767735336243266456152424263655442375437323425146317632655141433156665635625563452536534233413111
4452v3425411645566^>23>>545244^>35>>>>>>654776563^>55>463117221625365717275545124476464451442531715677337534443363223313661263112221651414352
43451425241436345251115v554236546325615v4632123721623v77673^>45>2324636^>47>>>568835776177657563335176174462444145653231645541641352324222432
512344115556454364551122335531173547353527264165771324616431564v5435216143433v624143426673724271674725735171453732677454152514121553642514342
2453v5631545534511^62451241637^565614543322125441^7121477616833536676467486356424113643275743254655271257165273233333713436363316152215525412
5251v>43>6654^>25>>4531v>55>>>>25774344v>62>>>>>>>476v38757^36765868875^454216857288228327638844315166557547372743531744154253555366125322251
24222615v4345^423164252216377734341337537542778666575v>32>>>584v>53>>>>>31438v>83>>>>15875143615383175544722257361643371324456124635121426143
355256

In [36]:
IN18 = !cat aoc/2023/day18

BIN18 = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)""".split("\n")

In [37]:
from collections import defaultdict

instr = list(map(str.split, IN18))

move = {
    "R": lambda pt, k=1: (pt[0], pt[1] + k),
    "L": lambda pt, k=1: (pt[0], pt[1] - k),
    "D": lambda pt, k=1: (pt[0] + k, pt[1]),
    "U": lambda pt, k=1: (pt[0] - k, pt[1]),
}

def total_area1(instructions):
    point = (0,0)
    fig = {point}
    for d, n, _ in instructions:
        for _ in range(int(n)):
            point = move[d](point)
            fig.add(point)

    minx, maxx = (min(p[0] for p in fig), max(p[0] for p in fig))
    miny, maxy = (min(p[1] for p in fig), max(p[1] for p in fig))

    valid = lambda p: minx <= p[0] <= maxx and miny <= p[1] <= maxy
    neighbors = lambda x, y: {p for p in {(x-1, y), (x+1, y), (x, y-1), (x, y+1)} if valid(p)}

    border = (
        {(k, miny) for k in range(minx, maxx+1)} 
        |  {(k, maxy) for k in range(minx, maxx+1)} 
        | {(minx, k) for k in range(miny, maxy+1)} 
        | {(maxx, k) for k in range(miny, maxy+1)}
    )
    ext = border - fig
    q = list(ext)
    while q:
        p = q.pop()
        newp = neighbors(*p) - fig - ext
        ext |= newp
        q.extend(list(newp))
    
    return (maxx-minx+1)*(maxy-miny+1) - len(ext)

ans1 = total_area1(instr)

########################################

point = (0,0)
vs = [point]
for _, _, hcode in list(instr):
    n = int(hcode[2:7], 16)
    d = "RDLU"[int(hcode[7])]
    vs.append(move[d](vs[-1], int(n)))

# Shoelace formula
area = abs(sum((v[0]*w[1] - v[1]*w[0]) for v, w in zip(vs[:-1], vs[1:]))) / 2

circ = sum(abs(v[0] - w[0]) + abs(v[1] - w[1]) for v, w in zip(vs[:-1], vs[1:]))

# Pick's theorem again, except now we need to include the cells on the boundary (+circ)
ans2 = int(area + 1 + circ/2)
ans1, ans2

(50603, 96556251590677)

In [38]:
IN19 = !cat aoc/2023/day19
BIN19 = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""".split("\n")

In [43]:
from itertools import groupby
from operator import mul

inflows, inparts = [list(g) for _, g in groupby(IN19, bool) if _]

def parse_flow(f):
    f = f[:-1]
    name, rs = f.split("{")
    rules = [r.split(":") for r in rs.split(",")]
    return name, rules

parse_part = lambda p: eval(f"dict({p[1:-1]})")
    

flows = dict(parse_flow(f) for f in inflows)
qs = {name: [] for name in flows} | {"A": [], "R": []} | {'in': [parse_part(p) for p in inparts]}

def next_flow(rules, part):
    for r in rules:
        if len(r) == 1 or eval(r[0], {}, part):
            return r[-1]

def loop(flows, qs):
    while True:
        if not (name := next((name for name in flows if qs[name]), None)):
            return
    
        while qs[name]:
            p = qs[name].pop()
            nflow = next_flow(flows[name], p)
            qs[nflow].append(p)
        
        
loop(flows, qs)    
ans1 = sum(sum(p.values()) for p in qs["A"])


########################################

N = 4000
ALL = set(range(1, N+1))

def rule_range(r):
    var, sign, value = r[0], r[1], str(r[2:])
    if sign == "<":
        return {var: set(range(1, int(value)))}
    else:
        return {var: set(range(int(value)+1, N+1))}

def and_(cond1, cond2):
    result = {k:set(v) for k, v in cond1.items()}
    for v2, s2 in cond2.items():
        if v2 in result:
            result[v2] &= s2
        else:
            result[v2] = s2

    return result
        
def not_(rule_rng):
    return {k: ALL-v for k, v in rule_rng.items()}
    
def negate(rules):
    cond = UNIVERSAL_PRIOR
    for r in rules:
        cond = and_(cond, not_(rule_range(r[0])))
    
    return cond

def prev_gen(flow, rules, prior):
    for i, r in enumerate(rules):
        if r[-1] != flow:
            continue
            
        cond =  and_(prior, negate(rules[:i]))
        if len(r) > 1:
            cond = and_(cond, rule_range(r[0]))
        
        if all(cond.values()):
            yield cond

def reverse_loop(flows, cs):
    while True:
        if not (name := next((k for k, v in cs.items() if v and k != "in"), None)):
            return
    
        prior = cs[name].pop()   
        for src, rules in flows.items():
            for parent in prev_gen(name, rules, prior):   
                cs[src].append(parent)

UNIVERSAL_PRIOR = {var: set(range(1, N+1)) for var in 'xmas'}
                    
cs = {name:[] for name in flows} | {"A": [], "R": [UNIVERSAL_PRIOR]}

reverse_loop(flows, cs)

prod = lambda prior: reduce(mul, [len(v) for v in prior.values()], 1) 
ans2 = 4000**4 - sum(prod(x) for x in cs["in"])
ans1, ans2

(434147, 136146366355609)

In [44]:
IN20 = !cat aoc/2023/day20
BIN20 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""".split("\n")

In [45]:
from collections import Counter
conn = [r.split(" -> ") for r in IN20]

LOW, HIGH = 0, 1

inputs = defaultdict(list)
mod_inputs = defaultdict(list)
outputs = defaultdict(dict)
behavior = {}

def flip_flopper(mod):
    state = [LOW]
    def act(pulse):
        if pulse == LOW:
            state[0] = 1 - state[0]
            yield (mod, state[0])
    return act

def conjunct(mod):
    def act(_):
        if all(m==HIGH for m in inputs[mod]):
            yield (mod, LOW)
        else:
            yield (mod, HIGH)
    return act
            
actions = {
    '%': flip_flopper,
    '&': conjunct,
}
            
for src, tgts in [[c[0], c[1].split(", ")] for c in conn]:
    if not src[0].isalnum():
        t, src = src[0], src[1:]
        behavior[src] = actions[t](src)

    for t in tgts:
        mod_inputs[t].append(src)
        inputs[t].append(LOW)
        outputs[src][t] = len(inputs[t]) - 1

lows = defaultdict(list)
        
def process_events(queue, c):
    c["press"] += 1
    while queue:
        module, pulse = queue.pop(0)

        for out, pos in outputs[module].items():
            inputs[out][pos] = pulse
            if out in {'rd', 'bt', 'fv', 'pr'} and pulse == LOW:
                lows[out].append(c["press"])
            c[pulse] += 1
            if action := behavior.get(out):
                queue.extend(action(pulse))      

c = Counter()
rec_inputs = defaultdict(list)
for _ in range(10000):
    process_events([("broadcaster", LOW)], c)
            
           
    
ans1 = c[LOW] * c[HIGH]

ans1

74800683034

In [46]:
lows

defaultdict(list,
            {'pr': [3793, 7586],
             'rd': [3911, 7822],
             'bt': [3917, 7834],
             'fv': [3929, 7858]})

In [47]:
ans2 = 3793 * 3911 * 3917 * 3929
ans2

228300182686739

In [48]:
IN21 = !cat aoc/2023/day21
BIN21 = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........""".split("\n")

In [49]:
from functools import lru_cache
garden = IN21

M = len(garden)
N = len(garden[0])

valid = lambda p: 0 <= p[0] <= M-1 and 0 <= p[1] <= N-1 and garden[p[0]][p[1]] != "#"
neighbors = lru_cache(lambda x, y: {p for p in {(x-1, y), (x+1, y), (x, y-1), (x, y+1)} if valid(p)})

start = None
for i, row in enumerate(garden):
    for j, char in enumerate(row):
        if char == "S":
            start = (i, j)

def reach(state, steps, nbfunc=neighbors):
    accum = [state, set()]
     
    for tick in range(1, steps+1):
        seen = accum[tick % 2] | set()
        next_state = set()
        
        for i, j in state:
            imod, jmod = i % M, j % N
            idiv, jdiv = i//M * M, j//N * N
            
            for ni, nj in nbfunc(imod, jmod):
                np = (idiv + ni, jdiv + nj)
                if np not in seen:
                    next_state.add(np)
                    seen.add(np)
                    
        accum[tick % 2] = seen
        state = next_state
    
    return len(accum[steps % 2])

ans1 = reach({start}, 64)

val = lambda i, j: garden[i % M][j % N]

nbs = {}
def neighbors2(x, y):
    if ns := nbs.get((x, y)):
            return ns
    return nbs.setdefault((x, y), {p for p in {(x-1, y), (x+1, y), (x, y-1), (x, y+1)} if garden[p[0] % M][p[1] % N] != "#"})

C = 8
cover = [reach({start}, 65 + M*k, nbfunc=neighbors2) for k in range(1, C)]
cover, diff(cover), diff(diff(cover))

k = (26501365 - 65)//M

ans2 = 34700 + (k-1)*61515 + (k-1)*(k-2)*30700//2

ans1, ans2

(3770, 628206330073385)

In [50]:
IN22 = !cat aoc/2023/day22

BIN22 = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9""".split("\n")

In [51]:
from collections import defaultdict
bricks = [list(map(int, re.split("[,~]", r))) for r in IN22]
xy = lambda x1, y1, x2, y2: [(i, j) for i in range(x1, x2+1) for j in range(y1, y2+1)]

def get_stacking(bricks):
    stacks = defaultdict(list)
    n = len(bricks)
    figs = [None] * n
    zs = [None] * n
    
    for i, (x1, y1, z1, x2, y2, z2) in enumerate(bricks):
        figs[i] = xy(x1, y1, x2, y2)
        zs[i] = (z1, z2)
        for p in figs[i]:
            stacks[p].append(i)
    
    for p, s in stacks.items():
        s.sort(key=lambda i: min(zs[i]))
        
    height = [None] * n
    
    support = {}

    for i in sorted(range(n), key=lambda k: min(zs[k])):
        zadd = abs(zs[i][0] - zs[i][1]) + 1
        top_height = zadd
        
        for p in figs[i]:
            if idx := stacks[p].index(i):
                j = stacks[p][idx-1]
            
                forced = height[j] + zadd
            
                if forced > top_height:
                    top_height = forced
                    support[i] = [j]
                elif forced == top_height:
                    support[i].append(j)

        height[i] = top_height
    
    singles = {v[0] for v in support.values() if len(set(v)) <= 1}
    return singles, support, height
    
    
singles, support, height = get_stacking(bricks)

ans1 = len(bricks) - len(singles)

########################################

ans2 = 0
support_sets = {k:set(v) for k, v in support.items()}
for b in range(len(bricks)):
    falling = {b}
    while True:
        for k, v in support_sets.items():
            if v <= falling and k not in falling:
                falling.add(k)
                break
        else:
            break
    ans2 += len(falling) - 1

ans1, ans2

(459, 75784)

In [52]:
IN23 = !cat aoc/2023/day23

BIN23 = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#""".split("\n")

In [53]:
fig = IN23
def desc(i, j):
    if fig[i][j] == "#":
        return set()
    if fig[i][j] == ">":
        return {(i, j+1)}
    if fig[i][j] == "<":
        return {(i, j-1)}
    if fig[i][j] == "v":
        return {(i+1, j)}
    if fig[i][j] == "^":
        return {(i-1, j)}
    
    return {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}

def desc2(i, j):
    if fig[i][j] == "#":
        return set()
    
    return {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}

def valid_desc(p, seen):
    return (p not in seen) and (0<=p[0]<len(fig) and 0<=p[1]<len(fig[0])) and fig[p[0]][p[1]] != "#"

def all_paths():
    start = (0, fig[0].index("."))
    q = [[start]]
    
    while q:
        path = q.pop()
        seen = set(path)
        nxt = [p for p in desc(*path[-1]) if valid_desc(p, seen)]
        while len(nxt) == 1:
            path.append(nxt[0])
            seen.add(nxt[0])
            nxt = [p for p in desc(*path[-1]) if valid_desc(p, seen)]
            
        if not nxt:
            if path[-1][0] == len(fig) - 1:
                yield path

        for p in nxt:
            q.append(path + [p])

        
ans1 = max(len(pp)-1 for pp in all_paths())

########################################


from itertools import groupby
nbs = {}
for i in range(len(fig)):
    for j in range(len(fig[0])):
        nbs[(i, j)] = [p for p in desc2(i, j) if valid_desc(p, set())]

degs = {}
for i, g in groupby(sorted(nbs, key=lambda v: len(nbs[v])), key=lambda v: len(nbs[v])):
    degs[i] = list(g)

# Construct a reduced graph with only the start, end, and intersections (36 vertices total)
V = degs[1] + degs[3] + degs[4]
E = {}
reduced_nbs = defaultdict(set)
for v in V:
    for n in nbs[v]:
        p = [v, n]
        while p[-1] not in V:
            p.append(next(n1 for n1 in nbs[p[-1]] if n1 != p[-2]))

        E[(p[0], p[-1])] = len(p)-1
        reduced_nbs[p[0]].add(p[-1])
        reduced_nbs[p[-1]].add(p[0])

        
from heapq import heappush, heappop

def bfs(start, goal):
    
    best = {v:-100000 for v in V}
    best[start] = 0

    q = [[0, start, set()]]

    unchanged = 0
    while q:
        # The idea is that trying all possible paths will be super-slow, but our heap heuristic
        # will find the longest path pretty quickly (in under 10K loops, I think, but why not keep going)
        # and then it will stay constant for a long time, 
        unchanged += 1
        if not unchanged % 1000000:
            print(f"{unchanged}: {best[goal]}")
        if unchanged > 10000000:
            return best[goal]
            
        cost_to_v1, v1, path_to_v1 = heappop(q)

        for neighbor in reduced_nbs[v1] - path_to_v1:
            curr = -cost_to_v1 + E[(v1, neighbor)]
            if curr > best[neighbor]:
                if neighbor == goal:
                    unchanged = 0
                    
                best[neighbor] = curr

            heappush(q, [-curr, neighbor, path_to_v1 | {v1}])

    raise Exception("failure")
    
ans2 = bfs(V[0], V[1])

ans1, ans2

1000000: 6514
2000000: 6514
3000000: 6514
4000000: 6514
5000000: 6514
6000000: 6514
7000000: 6514
8000000: 6514
9000000: 6514
10000000: 6514


(2110, 6514)

In [54]:
IN24 = !cat aoc/2023/day24

BIN24 = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3""".split("\n")


In [55]:
def parse_row(r):
    return [list(map(int, p.split(", "))) for p in r.split(" @ ")]

rows = [parse_row(r) for r in IN24]
n = len(rows)

def intersect(px, py, vx, vy, qx, qy, wx, wy):
    denom = wx * vy - wy * vx
    udiff = px * vy - py * vx - qx * vy + qy * vx
    tdiff = -qx * wy + qy * wx + px * wy - py * wx
    if denom == 0 or tdiff / denom < 0 or udiff / denom < 0:
        return []
    
    u = udiff / denom
    return [qx + u * wx, qy + u * wy]

def intersections_in_range(rows, minx, maxx, miny, maxy):
    result = []
    for i in range(len(rows) - 1):
        (px, py, _), (vx, vy, _) = rows[i]
        for j in range(i+1, n):
            (qx, qy, _), (wx, wy, _) = rows[j]
            if isect := intersect(px, py, vx, vy, qx, qy, wx, wy):
                if minx <= isect[0] <= maxx and miny <= isect[1] <= maxy:
                    result.append(isect)
                    
    return result

ans1 = len(intersections_in_range(rows, 200000000000000, 400000000000000, 200000000000000, 400000000000000))

########################################

import numpy as np

nprows = [(np.array(p), np.array(v)/v[2]) for p, v in rows]
# reparametrize each line with z instead of t, so the last intercept/slope coeffs are always 0/1,
# and we can just drop them. Also scale down the ps to avoid float weirdness
zrows = [[1e-13*(p - p[2]*vnorm)[:2], vnorm[:2]] for p, vnorm in nprows]

# (x, y) = pnorm + z * vnorm
# intersection in 2D can only lift to intersection in 3D if z is the same,
# and conversely, any intersection in 3D must lead to an intersection in 2D with same z,
# so if (x, y) = X + z * V is the line that intersects all of them, then
# (X - pnorm) = z * (vnorm - V)
# (X[0] - pn[0])(V[1] - vn[1]) = (X[1] - pn[1])(V[0] - vn[0])
# X[0]V[1] - pn[0]V[1] - X[0]vn[1] + pn[0]vn[1] = X[1]V[0] - pn[1]V[0] - X[1]vn[0] + pn[1]vn[0]
# subtract the same eqn but for the next pk, vk, so we lose the non-linear terms.
#
# There's some trivial linear algebra here that I can't quite express through numpy.
coeff = lambda p1, v1, p2, v2: [-v1[1] + v2[1], v1[0] - v2[0], p1[1] - p2[1], -p1[0] + p2[0]]
beta = lambda p1, v1, p2, v2: [p1[1]*v1[0] - p1[0]*v1[1] - p2[1]*v2[0] + p2[0]*v2[1]]

def solve_X_V(rownums):
    # each pair of rows gives us one equation in 4 unknowns (X[0], X[1], V[0], V[1]),
    # so make 4 eqns from 5 rows
    a = lambda n: coeff(*zrows[rownums[0]], *zrows[rownums[n]])
    b = lambda n: beta(*zrows[rownums[0]], *zrows[rownums[n]])
    
    A = np.array([a(j) for j in range(1, 5)])
    B = np.array([b(j) for j in range(1, 5)])

    return np.linalg.solve(A, B)
    

sol1 = solve_X_V([0, 1, 2, 3, 4])

# Try a totally different set of 5 rows. This should come out the same if the problem is solvable
sol2 = solve_X_V([10, 11, 12, 13, 14])

X, V = sol1[:2,0], sol1[2:,0]

# 90 makes V into a nice integer array, but we don't know if it's the right scale
VV = np.concatenate([90 * V, np.array([90])])

# so we generate all the times as solutions to
# p1 + t1 * v1 + k1 * VV = p2 + t2 * v2
# (ie, [v1, -v2, VV] * (t1, t2, k1) = p2 - p1)
# If VV has the right scale, then the ks and the ts will have the same range,
# since they are just a shift away
def get_timing():
    ts = [None] * n
    ks = [0] + [None] * (n-1)

    for i in range(1, n):
        p1, v1 = rows[0]
        p2, v2 = rows[i]
        t1, t2, k = np.linalg.solve(np.array([v1, -np.array(v2), VV]).transpose(), np.array(p2)-np.array(p1))

        ts[0] = t1
        ts[i] = t2
        ks[i] = k

    ratio = (max(ts) - min(ts)) / (max(ks) - min(ks))
    return ratio, ts, ks

while True:
    ratio, ts, ks = get_timing()
    if abs(ratio - 1) < 0.001:
        break
    else:
        VV /= ratio

p1, v1 = rows[0]
# The cross between our VV line and rows[0] happens at time ts[0]
cross = np.array(p1) + ts[0] * np.array(v1)
# So just figure out where VV started
start = cross - ts[0] * VV
ans2 = int(start.sum())

ans1, ans2

(12938, 976976197397181)

In [56]:
IN25 = !cat aoc/2023/day25

BIN25 = """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr""".split("\n")

In [57]:
rows = IN25
nbs = defaultdict(set)
for r in rows:
    x, y0 = r.split(": ")
    ys = y0.split()
    for y in ys:
        nbs[x].add(y)
        nbs[y].add(x)
        
start = list(nbs.keys())[0]
part1 = {start}
part2 = set(nbs) - part1

def get_cut(p1, p2):
    c = set()
    for v in p1:
        c |= {(v, n) for n in nbs[v] - p1}
    return c

cut = get_cut(part1, part2)

while True:
    if len(cut) <= 3:
        break
    
    
    if len(part2) > len(nbs)/3:
        pfrom, pto = part2, part1
    elif len(part1) > len(nbs)/3:
        pfrom, pto = part1, part2
    else:
        pfrom, pto = part2, part1
        
    b = list(sorted(pfrom, key=lambda b: len(nbs[b] & pfrom) - len(nbs[b] & pto)))[0]
    pfrom.remove(b)
    pto.add(b)
    cut = get_cut(part1, part2)

ans1 = len(part1) * len(part2)

ans1

555856